In [3]:
from stylistic import StyleEvaluator
import pandas as pd
import json
from sklearn.decomposition import PCA
import sklearn
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split

# Original dataset if you dont have PCA data
data = json.load(open(r'preprocess\experiments\data\pairwise_evaluation_w_embeddings.json'))
df = pd.DataFrame(data)

Logistic Regression Line function

In [5]:
def logistic_regression_line(x,y):
    model = LogisticRegression()
    X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    model.fit(X_train,Y_train)
    w1,w2 = model.coef_.T
    intercept = model.intercept_[0]
    c = -intercept/w2
    m = -w1/w2
    return c,m

Run if you dont have stylistic metrics data

In [ ]:
for i,datum in enumerate(data):
    print('{}/{}'.format(i, len(data)))
    scorer =StyleEvaluator()
    style_scores = scorer.default(datum['article_text'],datum['text-davinci-002_summary'])
    datum['readability'] = style_scores['readability']
    datum['formality'] = style_scores['formality']
    datum['sentiment'] = style_scores['sentiment']
    datum['faithfulness'] = style_scores['faithfulness']
    datum['length'] = style_scores['length']


Encode readability score below 0.5 as not-readable and above as readable

In [8]:
df = pd.DataFrame(data)
for index,datum in df.iterrows():
    if datum['readability']>=50.0:
        df.at[index,'readability']=1
    else:
        df.at[index,'readability']=0

X,Y for PCA 

In [9]:
X = df.drop(columns=['overall_writer_better','full_embedding','writer_summary_embedding','llm_summary_embedding','informative_writer_better','text-davinci-002_summary','writer_summary','article_text','evaluator_id','writer_id','article_id','readability'])
y = df['readability']

Standardize feature values

In [10]:
X_scaled = X.copy()  
for column in X_scaled.columns: 
    X_scaled[column] = (X_scaled[column] - X_scaled[column].mean()) / X_scaled[column].std()   

PCA

In [11]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

In [12]:
c,m = logistic_regression_line(X_pca,y)
print(c,m)

[-1.50590381] [0.21562842]
